Import Libraries

In [ ]:
from keras.utils import get_file
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras import datasets, layers, models,losses
from keras.optimizers import Adam
import tarfile
from nltk.stem.porter import PorterStemmer 
import os
import re
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Download zip data file and extract data

In [ ]:
data_dir = get_file('aclImdb_v1.tar.gz', 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz', cache_subdir = "datasets",hash_algorithm = "auto", extract = True, archive_format = "auto")

my_tar = tarfile.open(data_dir)
my_tar.extractall('./data/') # specify which folder to extract to
my_tar.close()

Read train data while preprocessing and removing stop words

In [ ]:
data_folder="/train"
imdb_folder = "/aclImdb"
train_reviews = []
train_labels = []
ps = PorterStemmer()
for index,sentiment in enumerate(["/neg/", "/pos/"]):
    path = "./data"+imdb_folder + data_folder + sentiment
    for filename in sorted(os.listdir(path)):
        i=i+1
        with open(path + filename, 'r') as f:
            review = f.read()
            review = review.lower()
            review = review.rstrip()
            review = review.replace("<br />", " ")
            review = re.sub(r"[^a-z ]", " ", review) 
            review = re.sub(r" +", " ", review)
            word_tokens = word_tokenize(review) 
            #remove stop words
            filtered_sentence = [w for w in word_tokens if not w in stop_words] 
            filtered_sentence = [] 
            for w in word_tokens: 
                if w not in stop_words: 
                    filtered_sentence.append(w) 
            review=filtered_sentence
            train_reviews.append(review)
            
            label = [0, 0]
            label[index] = 1
            train_labels.append(label)

Create array for reveiws and labels for train data

In [ ]:
train_reveiws=np.array(train_reviews)
print(train_reveiws.shape)
train_labels=np.array(train_labels)
print(train_labels.shape)

(25000,)
(25000, 2)


In [ ]:
train_reveiws[1]

['airport',
 'starts',
 'brand',
 'new',
 'luxury',
 'plane',
 'loaded',
 'valuable',
 'paintings',
 'belonging',
 'rich',
 'businessman',
 'philip',
 'stevens',
 'james',
 'stewart',
 'flying',
 'bunch',
 'vip',
 'estate',
 'preparation',
 'opened',
 'public',
 'museum',
 'also',
 'board',
 'stevens',
 'daughter',
 'julie',
 'kathleen',
 'quinlan',
 'son',
 'luxury',
 'jetliner',
 'takes',
 'planned',
 'mid',
 'air',
 'plane',
 'hi',
 'jacked',
 'co',
 'pilot',
 'chambers',
 'robert',
 'foxworth',
 'two',
 'accomplice',
 'banker',
 'monte',
 'markham',
 'wilson',
 'michael',
 'pataki',
 'knock',
 'passengers',
 'crew',
 'sleeping',
 'gas',
 'plan',
 'steal',
 'valuable',
 'cargo',
 'land',
 'disused',
 'plane',
 'strip',
 'isolated',
 'island',
 'making',
 'descent',
 'chambers',
 'almost',
 'hits',
 'oil',
 'rig',
 'ocean',
 'loses',
 'control',
 'plane',
 'sending',
 'crashing',
 'sea',
 'sinks',
 'bottom',
 'right',
 'bang',
 'middle',
 'bermuda',
 'triangle',
 'air',
 'short',
 's

Read test data while preprocessing and removing stop words

In [ ]:
data_folder="/test"
imdb_folder = "/aclImdb"
test_reviews = []
test_labels = []
ps = PorterStemmer()
for index,sentiment in enumerate(["/neg/", "/pos/"]):
    path = "./data"+imdb_folder + data_folder + sentiment
    for filename in sorted(os.listdir(path)):
        with open(path + filename, 'r') as f:
            review = f.read()
            review = review.lower()
            review = review.rstrip()
            review = review.replace("<br />", " ")
            review = re.sub(r"[^a-z ]", " ", review) 
            review = re.sub(r" +", " ", review)
            word_tokens = word_tokenize(review) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words] 
            filtered_sentence = [] 
            for w in word_tokens: 
                if w not in stop_words: 
                    filtered_sentence.append(w) 
            review=filtered_sentence
            test_reviews.append(review)
            
            label = [0, 0]
            label[index] = 1
            test_labels.append(label)

In [ ]:
print(test_reviews[0])

['mr', 'costner', 'dragged', 'movie', 'far', 'longer', 'necessary', 'aside', 'terrific', 'sea', 'rescue', 'sequences', 'care', 'characters', 'us', 'ghosts', 'closet', 'costner', 'character', 'realized', 'early', 'forgotten', 'much', 'later', 'time', 'care', 'character', 'really', 'care', 'cocky', 'overconfident', 'ashton', 'kutcher', 'problem', 'comes', 'kid', 'thinks', 'better', 'anyone', 'else', 'around', 'shows', 'signs', 'cluttered', 'closet', 'obstacle', 'appears', 'winning', 'costner', 'finally', 'well', 'past', 'half', 'way', 'point', 'stinker', 'costner', 'tells', 'us', 'kutcher', 'ghosts', 'told', 'kutcher', 'driven', 'best', 'prior', 'inkling', 'foreshadowing', 'magic', 'could', 'keep', 'turning', 'hour']


Create array for reveiws and labels for test data

In [ ]:
test_reveiws=np.array(test_reviews)
print(test_reveiws.shape)
test_labels=np.array(test_labels)
print(test_labels.shape)

(25000,)
(25000, 2)


Get total number of unique words in train data to apply one hot encoding

In [ ]:
all_words = []
for review in train_reveiws:
    for word in review:
        all_words.append(word)
unique_words = set(all_words)
vocab_length=len(unique_words)+50
print(len(unique_words))

73115


Apply one hot encoding to the train data

In [ ]:
train_enc_review=[]
for review in train_reveiws:
    enc_review=[]
    enc_word = [one_hot(word, vocab_length) for word in review]
    for x in enc_word:
        for y in x:
            enc_review.append(y)
    train_enc_review.append(enc_review)
enc_review=np.array(enc_review)
print(enc_review.shape)
train_enc_review=np.array(train_enc_review)
print(train_enc_review.shape)

(91,)
(25000,)


In [ ]:
print(train_enc_review[0])
print(train_enc_review[1])

[43691, 32618, 3757, 28066, 11019, 68867, 27522, 66235, 65228, 35663, 17511, 66764, 51285, 63000, 68362, 28710, 61453, 16518, 48682, 16796, 6824, 28009, 6269, 32356, 17511, 32537, 61546, 382, 60653, 28405, 54582, 35799, 72808, 6760, 28710, 31257, 24806, 12498, 1500, 64716, 1067, 70158, 50359, 27284, 38068, 37899, 59648, 37240, 1353, 23264, 55504, 53538, 45445, 20827, 42857, 53538, 38882, 4818, 2553, 45671, 49159, 40265, 4074]
[47857, 68867, 25429, 18896, 60807, 70285, 40436, 14302, 16276, 29263, 6539, 20797, 54886, 44928, 49678, 66840, 59608, 15318, 6072, 27647, 10746, 6758, 55332, 47193, 29609, 8208, 44928, 51702, 27492, 2018, 11872, 62920, 60807, 62395, 17724, 43770, 47479, 17993, 70285, 60598, 37942, 47510, 5364, 21595, 26023, 72194, 29430, 12753, 11628, 35051, 34052, 9294, 31479, 38156, 52537, 29332, 50318, 20360, 17881, 11206, 72788, 14302, 62592, 62292, 45910, 70285, 63340, 12706, 6030, 54582, 55297, 21595, 31920, 63930, 70790, 49363, 61234, 3238, 43492, 70285, 7970, 69800, 69452

Padding the train data to make it of similar length

In [ ]:
padded_inputs = pad_sequences(train_enc_review, padding="post",maxlen=2000)
print(padded_inputs[0].shape)
print(padded_inputs[1].shape)
print(padded_inputs.shape)

(2000,)
(2000,)
(25000, 2000)


Define the model


In [ ]:
max_len=2000
model = Sequential()
model.add(layers.Embedding(vocab_length,20,input_length=max_len))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 2000, 20)          1463300   
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 2000, 32)          1952      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 1000, 32)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 32000)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 250)               8000250   
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 502       
Total params: 9,466,004
Trainable params: 9,466,004
Non-trainable params: 0
___________________________________________

In [ ]:
print(padded_inputs.shape)
print(train_labels.shape)

(25000, 2000)
(25000, 2)


Compile and train the model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded_inputs,train_labels,epochs=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
25000/25000 [==============================] - 159s 6ms/step - loss: 0.3742 - accuracy: 0.8203
Epoch 2/10
25000/25000 [==============================] - 157s 6ms/step - loss: 0.1437 - accuracy: 0.9490
Epoch 3/10
25000/25000 [==============================] - 157s 6ms/step - loss: 0.0579 - accuracy: 0.9815
Epoch 4/10
25000/25000 [==============================] - 160s 6ms/step - loss: 0.0202 - accuracy: 0.9949
Epoch 5/10
25000/25000 [==============================] - 155s 6ms/step - loss: 0.0083 - accuracy: 0.9977
Epoch 6/10
25000/25000 [==============================] - 157s 6ms/step - loss: 0.0120 - accuracy: 0.9959
Epoch 7/10
25000/25000 [==============================] - 157s 6ms/step - loss: 0.0072 - accuracy: 0.9979
Epoch 8/10
25000/25000 [==============================] - 157s 6ms/step - loss: 0.0077 - accuracy: 0.9973
Epoch 9/10
25000/25000 [==============================] - 159s 6ms/step - loss: 0.0054 - accuracy: 0.9983
Epoch 10/10
25000/25000 [=====================

Apply one hot encoding to the test data

In [ ]:
test_enc_review=[]
for review in test_reveiws:
    enc_review=[]
    enc_word = [one_hot(word, vocab_length) for word in review]
    for x in enc_word:
        for y in x:
            enc_review.append(y)
    test_enc_review.append(enc_review)
enc_review=np.array(enc_review)
print(enc_review.shape)
test_enc_review=np.array(test_enc_review)
print(test_enc_review.shape)

(119,)
(25000,)


In [ ]:
print(test_enc_review[0])

[59262, 5707, 27543, 8802, 38419, 60333, 36193, 39183, 65228, 69452, 56711, 65671, 41448, 39670, 53746, 21542, 41213, 5707, 3152, 9763, 18859, 58719, 36703, 69468, 61546, 41448, 3152, 42844, 41448, 2879, 6580, 70107, 54670, 51054, 16384, 7651, 69990, 59648, 61224, 4065, 25368, 34366, 24710, 42050, 41213, 46254, 13506, 10365, 5707, 43631, 34492, 44279, 2048, 26583, 67819, 45758, 5707, 3035, 53746, 54670, 21542, 11179, 54670, 31561, 29971, 35278, 11020, 61071, 48441, 71034, 17113, 67374, 14839]


Padding the test data to make it of similar length

In [ ]:
padded_test = pad_sequences(test_enc_review, padding="post",maxlen=2000)
print(padded_test[0].shape)
print(padded_test[1].shape)
print(padded_test.shape)

(2000,)
(2000,)
(25000, 2000)


Evaluate the model for test data

In [ ]:
scores = model.evaluate(padded_test, test_labels, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.26%
